<a href="https://colab.research.google.com/github/alessandronicolini/IncrementalLearning/blob/main/baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import project files files from github 


In [1]:
# upload work files from your git hub repository
import sys

!git clone https://github.com/alessandronicolini/IncrementalLearning.git # clone proj repository
!rm -rf IncrementalLearning/README.md 
!rm -rf IncrementalLearning/baselines.ipynb

path = 'IncrementalLearning/'
if path not in sys.path:
    sys.path.append('IncrementalLearning/')

!pip3 install import_ipynb

fatal: destination path 'IncrementalLearning' already exists and is not an empty directory.


import libraries


In [2]:
import torch
import os
import torch.nn as nn
from torch.utils.data import Subset, DataLoader
import torch.optim as optim
from torchvision import transforms
import numpy as np
import import_ipynb
import copy
# project classes --------------------------------------------------------------
from cifar100 import ilCIFAR100
from resnet_cifar import resnet32
# from info_recorder.info_log import InfoLog
# from models_benchmark.benchmark import Benchmark

importing Jupyter notebook from cifar100.ipynb
Files already downloaded and verified
[60, 34, 84, 67, 85, 44, 18, 48, 1, 47, 61, 35, 82, 58, 76, 29, 71, 0, 79, 93, 56, 90, 20, 43, 26, 7, 73, 25, 9, 65, 95, 51, 11, 2, 74, 28, 96, 27, 99, 64, 70, 42, 62, 8, 98, 77, 39, 88, 10, 94, 3, 52, 68, 32, 5, 72, 38, 75, 69, 30, 40, 41, 24, 55, 91, 45, 12, 16, 22, 53, 63, 57, 31, 33, 21, 83, 49, 81, 59, 78, 97, 19, 46, 17, 36, 87, 6, 13, 14, 89, 80, 23, 86, 37, 54, 92, 50, 66, 15, 4]
0
1
2
3
4
5
6
7
8
9
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


Initial setup

In [13]:
# Define params

NUM_RUN = 2
SEEDS = [1,2,3]
NUM_EPOCHS = 70
BATCH_SIZE = 128
CLASSES_PER_BATCH = 10
DEVICE = 'cuda'
NUM_BATCHES=10
WEIGHT_DECAY = 1e-5
LR = 2
MOMENTUM = 0.9
MILESTONES = [49, 63]
GAMMA = 0.2
randomseed=10
SAVINGS_DIR = "savings"
# Create savings folder
try:
    os.mkdir(SAVINGS_DIR)
except FileExistsError:
    pass

Define transforms

In [4]:
# Define transformations for training
train_transform = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Define transformations for evaluation
test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))                                    
])

Prepare dataloders for each train, val test subset of the original dataset

FINE TUNING

In [20]:
def trainmodel(model,trainloader,valloader,learningrate,weightdecay,numepochs,milestones,gamma,criterion,optimizer,scheduler,dictionary):
  model=model.to(DEVICE)
  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0
  for epoch in range(numepochs):
    #print(epoch)
    for phase in ['train','val']:
      if phase=='train':
        model.train()
        loader=trainloader
      else:
        model.eval()
        loader=valloader
      running_loss=0.0
      running_corrects=0
      num_elements=0
      for inputs,labels in loader:
        labels = torch.tensor([torch.tensor(diz[c.item()]) for c in labels])
        num_elements+=len(inputs)
        inputs=inputs.to(DEVICE)
        labels=labels.to(DEVICE)
        optimizer.zero_grad()
        with torch.set_grad_enabled(phase == 'train'):
          outputs=model(inputs)
          labels_encoded=torch.eye(100)[labels] #CAMBIARE ONE_HOT
          labels_encoded=labels_encoded.to(DEVICE)
          _,preds=torch.max(outputs,1)
          loss=criterion(outputs,labels_encoded)
          if phase=='train':
            loss.backward()
            optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
      if phase == 'train':
        scheduler.step()

      epoch_loss = running_loss / num_elements
      epoch_acc = running_corrects.double() / num_elements
      
            # deep copy the model
      if phase == 'val' and epoch_acc > best_acc:
          best_acc = epoch_acc
          best_model_wts = copy.deepcopy(model.state_dict())
          #print(best_acc)
    # load best model weights
  model.load_state_dict(best_model_wts)
  return model


In [23]:
original_training_set = ilCIFAR100(CLASSES_PER_BATCH,randomseed)
original_test_set = ilCIFAR100(CLASSES_PER_BATCH,randomseed, train=False)
train_indices, val_indices = original_training_set.get_train_val(0.1)
test_indices = original_test_set.get_batch_indexes()
current_test_indexes=[]
model=resnet32(num_classes=100)
diz = original_training_set.get_dict()
acc=[]
criterion=nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=MILESTONES, gamma=GAMMA)
for i in range(NUM_BATCHES):
  print(i)
  train_dataset=Subset(original_training_set,train_indices[i])
  val_dataset=Subset(original_training_set,val_indices[i])
  current_test_indexes+=test_indices[i].tolist()
  test_dataset=Subset(original_test_set,current_test_indexes)
  train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
  val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
  test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)
  net=trainmodel(model,train_dataloader,val_dataloader,LR,WEIGHT_DECAY,NUM_EPOCHS,MILESTONES,GAMMA,criterion,optimizer,scheduler,diz)
  net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
  net.train(False) # Set Network to evaluation mode
  running_corrects = 0
  y_true = []
  y_pred = []
  for inputs, labels in test_dataloader:
      labels = torch.tensor([torch.tensor(diz[c.item()]) for c in labels])
      inputs = inputs.to(DEVICE)
      labels = labels.to(DEVICE)
      outputs = net(inputs)
      _, preds = torch.max(outputs.data, 1)
      running_corrects += torch.sum(preds == labels.data).data.item()
      labels=labels.detach().cpu().numpy()
      labels=labels.tolist()
      y_true.extend(labels)
      preds=preds.detach().cpu().numpy()
      y_pred_tmp = [p for p in preds]
      y_pred.extend(y_pred_tmp)

  # Calculate Accuracy
  accuracy = running_corrects / float(len(test_dataloader.dataset))
  print('Test Accuracy: %.2f' % (100.0 * accuracy))
  print('-' * 80)
  acc.append(accuracy)



Files already downloaded and verified
[73, 4, 54, 61, 99, 1, 26, 59, 62, 35, 83, 20, 98, 66, 91, 41, 9, 31, 46, 5, 53, 17, 77, 45, 48, 79, 36, 33, 58, 22, 38, 81, 78, 71, 30, 56, 75, 2, 37, 0, 15, 8, 12, 19, 34, 23, 49, 92, 88, 42, 90, 28, 27, 65, 87, 84, 96, 51, 32, 10, 14, 93, 52, 85, 95, 82, 64, 76, 80, 44, 57, 18, 11, 72, 68, 3, 24, 39, 55, 6, 47, 13, 63, 70, 7, 40, 25, 74, 89, 86, 29, 94, 67, 97, 69, 16, 43, 60, 50, 21]
Files already downloaded and verified
[73, 4, 54, 61, 99, 1, 26, 59, 62, 35, 83, 20, 98, 66, 91, 41, 9, 31, 46, 5, 53, 17, 77, 45, 48, 79, 36, 33, 58, 22, 38, 81, 78, 71, 30, 56, 75, 2, 37, 0, 15, 8, 12, 19, 34, 23, 49, 92, 88, 42, 90, 28, 27, 65, 87, 84, 96, 51, 32, 10, 14, 93, 52, 85, 95, 82, 64, 76, 80, 44, 57, 18, 11, 72, 68, 3, 24, 39, 55, 6, 47, 13, 63, 70, 7, 40, 25, 74, 89, 86, 29, 94, 67, 97, 69, 16, 43, 60, 50, 21]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'